In [1]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import random
import time
from IPython.display import clear_output

In [2]:
data = pd.read_csv('../dataset/raw/raw_paysim.csv')
data.columns= data.columns.str.lower()
data

,step,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isfraud,isflaggedfraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1577096,155,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,0.0
1577097,155,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,0.0
1577098,155,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,0.0
1577099,155,TRANSFER,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,0.0


In [3]:
data.dropna()

,step,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isfraud,isflaggedfraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1577095,155,TRANSFER,214977.94,C624724710,0.0,0.00,C1294409003,831919.08,1046897.02,0.0,0.0
1577096,155,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,0.0
1577097,155,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,0.0
1577098,155,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,0.0


In [4]:
data.rename(columns = {'isfraud':'EVENT_LABEL'}, inplace = True)

cols = data.columns.tolist()
cols = cols[0:9] + [cols[10]] + [cols[9]]
data = data[cols]

data = data.replace({'EVENT_LABEL': 0}, 'legit')
data = data.replace({'EVENT_LABEL': 1}, 'fraud')
data

,step,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isflaggedfraud,EVENT_LABEL
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,legit
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,legit
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,0.0,fraud
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,0.0,fraud
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,legit
...,...,...,...,...,...,...,...,...,...,...,...
1577096,155,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,legit
1577097,155,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,legit
1577098,155,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,legit
1577099,155,TRANSFER,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,legit


In [5]:
def isdigit(num):
    if num < 10:
        return '0' + str(num)
    else:
        return str(num)

In [6]:
event_timestamp = []
for i, row in data.iterrows():
    step = row['step']
    month = isdigit(step // 30 + 3)
    day = isdigit(30 if step % 30 == 0 else step % 30)
    hours = isdigit(random.randint(0, 23))
    minutes = isdigit(random.randint(0, 59))
    seconds = isdigit(random.randint(0, 59))
    timestamp = '2022-'+month+'-'+day+'T'+hours+':'+minutes+':'+seconds +'Z'
    event_timestamp.append(timestamp)
data['EVENT_TIMESTAMP'] = event_timestamp
data = data.drop('step', axis=1)
data

,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isflaggedfraud,EVENT_LABEL,EVENT_TIMESTAMP
0,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,legit,2022-03-01T04:25:56Z
1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,legit,2022-03-01T11:56:48Z
2,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,0.0,fraud,2022-03-01T00:03:27Z
3,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,0.0,fraud,2022-03-01T02:35:08Z
4,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,legit,2022-03-01T08:49:22Z
...,...,...,...,...,...,...,...,...,...,...,...
1577096,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,legit,2022-08-05T21:09:46Z
1577097,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,legit,2022-08-05T21:00:18Z
1577098,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,legit,2022-08-05T17:46:55Z
1577099,TRANSFER,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,legit,2022-08-05T08:12:30Z


In [7]:
event_id = [i for i in range(len(data))]
data['EVENT_ID'] = event_id

entity_id = [row['nameorig']+'-'+row['namedest'] for i,row in data.iterrows()]
data['ENTITY_ID'] = entity_id
data

,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isflaggedfraud,EVENT_LABEL,EVENT_TIMESTAMP,EVENT_ID,ENTITY_ID
0,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,legit,2022-03-01T04:25:56Z,0,C1231006815-M1979787155
1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,legit,2022-03-01T11:56:48Z,1,C1666544295-M2044282225
2,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,0.0,fraud,2022-03-01T00:03:27Z,2,C1305486145-C553264065
3,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,0.0,fraud,2022-03-01T02:35:08Z,3,C840083671-C38997010
4,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,legit,2022-03-01T08:49:22Z,4,C2048537720-M1230701703
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577096,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,legit,2022-08-05T21:09:46Z,1577096,C730444513-C814720871
1577097,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,legit,2022-08-05T21:00:18Z,1577097,C1893451872-C1661673055
1577098,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,legit,2022-08-05T17:46:55Z,1577098,C746221708-C1786396642
1577099,TRANSFER,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,legit,2022-08-05T08:12:30Z,1577099,C1834005599-C1985986684


In [8]:
#ENTITY_TYPE - Who is performing the activity, e.g. customer. 
#Currently, AFD only support one ENTITY_TYPE per EVENT_TYPE. That column should contain a single value.
#LABEL_TIMESTAMP - Timestamp of when the label is updated. LABEL_TIMESTAMP is required if EVENT_LABEL is included. 
#If you do not have that data, you can duplicate the EVENT_TIMESTAMP column and rename it as LABEL_TIMESTAMP.

entity_type = ['paysimentity' for i in range(len(data))]
data['ENTITY_TYPE'] = entity_type

label_timestamp = [row['EVENT_TIMESTAMP'] for i,row in data.iterrows()]
data['LABEL_TIMESTAMP'] = label_timestamp
data

,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isflaggedfraud,EVENT_LABEL,EVENT_TIMESTAMP,EVENT_ID,ENTITY_ID,ENTITY_TYPE,LABEL_TIMESTAMP
0,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,legit,2022-03-01T04:25:56Z,0,C1231006815-M1979787155,paysimentity,2022-03-01T04:25:56Z
1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,legit,2022-03-01T11:56:48Z,1,C1666544295-M2044282225,paysimentity,2022-03-01T11:56:48Z
2,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,0.0,fraud,2022-03-01T00:03:27Z,2,C1305486145-C553264065,paysimentity,2022-03-01T00:03:27Z
3,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,0.0,fraud,2022-03-01T02:35:08Z,3,C840083671-C38997010,paysimentity,2022-03-01T02:35:08Z
4,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,legit,2022-03-01T08:49:22Z,4,C2048537720-M1230701703,paysimentity,2022-03-01T08:49:22Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577096,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,legit,2022-08-05T21:09:46Z,1577096,C730444513-C814720871,paysimentity,2022-08-05T21:09:46Z
1577097,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,legit,2022-08-05T21:00:18Z,1577097,C1893451872-C1661673055,paysimentity,2022-08-05T21:00:18Z
1577098,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,legit,2022-08-05T17:46:55Z,1577098,C746221708-C1786396642,paysimentity,2022-08-05T17:46:55Z
1577099,TRANSFER,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,legit,2022-08-05T08:12:30Z,1577099,C1834005599-C1985986684,paysimentity,2022-08-05T08:12:30Z


In [10]:
data.to_csv('../dataset/prepared/prepared_paysim_tfi.csv', index=False)

In [11]:
s3 = boto3.resource('s3')
BUCKET = 'paysim-fraud-detection-bucket'

s3.Bucket(BUCKET).upload_file('../dataset/prepared/prepared_paysim_tfi.csv', 'prepared_paysim_tfi.csv')

In [12]:
fraudDetector = boto3.client('frauddetector')
            
response = fraudDetector.get_detectors()
print(response)

{'detectors': [], 'ResponseMetadata': {'RequestId': 'a6e0d50d-31dc-4f3f-aa3a-b488a2de255f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 09 Sep 2022 08:01:45 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '16', 'connection': 'keep-alive', 'x-amzn-requestid': 'a6e0d50d-31dc-4f3f-aa3a-b488a2de255f'}, 'RetryAttempts': 0}}


In [ ]:
fraudDetector.create_variable(
    name = 'amount1',
    variableType = 'PRICE',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)


fraudDetector.create_variable(
    name = 'isflaggedfraud1',
    variableType = 'CATEGORICAL',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
)   

fraudDetector.create_variable(
    name = 'namedest1',
    variableType = 'FREE_FORM_TEXT',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
)

fraudDetector.create_variable(
    name = 'nameorig1',
    variableType = 'FREE_FORM_TEXT',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
)


fraudDetector.create_variable(
    name = 'newbalancedest1',
    variableType = 'NUMERIC',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)

fraudDetector.create_variable(
    name = 'newbalanceorig1',
    variableType = 'NUMERIC',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)

fraudDetector.create_variable(
    name = 'oldbalancedest1',
    variableType = 'NUMERIC',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)

fraudDetector.create_variable(
    name = 'oldbalanceorg1',
    variableType = 'NUMERIC',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)

fraudDetector.create_variable(
    name = 'type1',
    variableType = 'PAYMENT_TYPE',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
)



In [ ]:
fraudDetector.put_entity_type(
   name = 'paysimentity',
)

In [ ]:
fraudDetector.put_label(
    name = 'fraud',
)

fraudDetector.put_label(
    name = 'legit',
)

In [ ]:
fraudDetector.put_event_type (
     name = 'paysim-event',
     eventVariables = ['amount', 'isflaggedfraud', 'namedest', 'nameorig','newbalancedest', 'newbalanceorig', 'oldbalancedest', 'oldbalanceorg', 'type'],
     labels = ['legit', 'fraud'],
     entityTypes = ['paysimentity'])

In [14]:
EVENT_TYPE = 'paysim-event'
S3_BUCKET = 'paysim-fraud-detection-bucket'
S3_FILE = 'prepared_paysim_tfi.csv'
ARN_ROLE = 'arn:aws:iam::235823010608:role/service-role/AmazonFraudDetector-DataAccessRole-1659514577130'
fraudDetector.create_batch_import_job(
    jobId = f'batch_import_{EVENT_TYPE}',
    inputPath = f's3://{S3_BUCKET}/{S3_FILE}',
    outputPath = f's3://{S3_BUCKET}',
    eventTypeName = EVENT_TYPE,
    iamRoleArn = ARN_ROLE
)

{'ResponseMetadata': {'RequestId': '97da858e-a11b-4c7e-b6bb-4ea83a44bbd1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 09 Sep 2022 08:01:56 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '97da858e-a11b-4c7e-b6bb-4ea83a44bbd1'},
  'RetryAttempts': 0}}

In [16]:
fraudDetector.create_model (
       modelId = 'paysim_tfi_fraud_detection_model',
       eventTypeName = 'paysim-event',
       modelType = 'TRANSACTION_FRAUD_INSIGHTS')

{'ResponseMetadata': {'RequestId': 'b2155a94-bb4e-4290-a21c-b69d1e3c59e1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 09 Sep 2022 08:46:46 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b2155a94-bb4e-4290-a21c-b69d1e3c59e1'},
  'RetryAttempts': 0}}

In [18]:
fraudDetector.create_model_version (
         modelId = 'paysim_tfi_fraud_detection_model',
         modelType = 'TRANSACTION_FRAUD_INSIGHTS',
         trainingDataSource = 'INGESTED_EVENTS',
         trainingDataSchema = {
            'modelVariables' : ['amount', 'isflaggedfraud', 'namedest', 'nameorig','newbalancedest', 'newbalanceorig', 'oldbalancedest', 'oldbalanceorg', 'type'],
            'labelSchema' : {
               'labelMapper' : {
                   'FRAUD' : ['fraud'],
                   'LEGIT' : ['legit']
                },
                'unlabeledEventsTreatment': 'IGNORE'
            }
         },
        ingestedEventsDetail={
            'ingestedEventsTimeWindow': {
                'startTime': data['EVENT_TIMESTAMP'].min(),
                'endTime': data['EVENT_TIMESTAMP'].max()
            }
        }        
)

{'modelId': 'paysim_tfi_fraud_detection_model',
 'modelType': 'TRANSACTION_FRAUD_INSIGHTS',
 'modelVersionNumber': '1.0',
 'status': 'TRAINING_IN_PROGRESS',
 'ResponseMetadata': {'RequestId': '1e7da9fe-59eb-4294-b132-ae4511392f50',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 09 Sep 2022 08:48:18 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '146',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1e7da9fe-59eb-4294-b132-ae4511392f50'},
  'RetryAttempts': 0}}